In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA A100-SXM4-40GB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install ultralytics
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 kB 12.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [ ]:
SEED = 42
BATCH_SIZE = 8
MODEL = "v2"

In [ ]:
if os.path.exists("../data/yolo"):
    shutil.rmtree("../data/yolo")

if not os.path.exists("../data/yolo/train"):
    os.makedirs("../data/yolo/train")

if not os.path.exists("../data/yolo/valid"):
    os.makedirs("../data/yolo/valid")

if not os.path.exists("../data/yolo/test"):
    os.makedirs("../data/yolo/test")

if not os.path.exists("../results"):
    os.makedirs("../results")

In [ ]:
def make_yolo_dataset(image_paths, txt_paths, type="train"):
    for image_path, txt_path in tqdm(zip(image_paths, txt_paths if not type == "test" else image_paths), total=len(image_paths)):
        source_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_height, image_width, _ = source_image.shape

        target_image_path = f"../data/yolo/{type}/{os.path.basename(image_path)}"
        cv2.imwrite(target_image_path, source_image)

        if type == "test":
            continue

        with open(txt_path, "r") as reader:
            yolo_labels = []
            for line in reader.readlines():
                line = list(map(float, line.strip().split(" ")))
                class_name = int(line[0])
                x_min, y_min = float(min(line[5], line[7])), float(min(line[6], line[8]))
                x_max, y_max = float(max(line[1], line[3])), float(max(line[2], line[4]))
                x, y = float(((x_min + x_max) / 2) / image_width), float(((y_min + y_max) / 2) / image_height)
                w, h = abs(x_max - x_min) / image_width, abs(y_max - y_min) / image_height
                yolo_labels.append(f"{class_name} {x} {y} {w} {h}")

        target_label_txt = f"../data/yolo/{type}/{os.path.basename(txt_path)}"
        with open(target_label_txt, "w") as writer:
            for yolo_label in yolo_labels:
                writer.write(f"{yolo_label}\n")

In [ ]:
image_paths = sorted(glob("/content/drive/MyDrive/df동아리/yolo이용 자동차 탐지/data/yolo/train/*.png"))
txt_paths = sorted(glob("/content/drive/MyDrive/df동아리/yolo이용 자동차 탐지/data/yolo/train/*.txt"))

train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.1, random_state=SEED)

make_yolo_dataset(train_images_paths, train_txt_paths, "train")
make_yolo_dataset(valid_images_paths, valid_txt_paths, "valid")
make_yolo_dataset(sorted(glob("/content/drive/MyDrive/df동아리/yolo이용 자동차 탐지/data/yolo/test/*.png")), None, "test")

100%|██████████| 3400/3400 [15:05<00:00,  3.76it/s]


In [ ]:
import os
import yaml

# 파일 경로
file_path = "/content/drive/MyDrive/df동아리/yolo이용 자동차 탐지/data/yolocustom.yaml"
directory = os.path.dirname(file_path)

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(directory):
    os.makedirs(directory)

# classes.txt 파일 읽기
with open("/content/drive/MyDrive/df동아리/yolo이용 자동차 탐지/data/classes.txt", "r") as reader:
    lines = reader.readlines()
    classes = [line.strip().split(",")[1] for line in lines]

# yaml 데이터 구성
yaml_data = {
    "names": classes,
    "nc": len(classes),
    "path": "/content/drive/MyDrive/df동아리/yolo이용 자동차 탐지/data/yolo/",
    "train": "train",
    "val": "valid",
    "test": "test"
}

# yaml 파일 작성
with open(file_path, "w") as writer:
    yaml.dump(yaml_data, writer)

In [ ]:
# # 필요한 라이브러리 설치
# pip install ultralytics

# # PyTorch와 torchvision 설치 (GPU 지원)
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# # Google Drive를 사용하여 파일을 불러오기 위한 라이브러리 설치
# pip install google-colab
# pip install gdown

In [3]:
%pip install ultralytics
%pip install scikit-learn

from google.colab import drive
drive.mount('/content/drive')

import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

SEED = 42
BATCH_SIZE = 8
MODEL = "v2"

Mounted at /content/drive


In [ ]:
#!cp -r /content/drive/MyDrive/df동아리/yolo이용\ 자동차\ 탐지/data /content/local_data

In [4]:
# YOLO 모델을 로드합니다. 여기서는 YOLOv8x 모델을 사용하고 있습니다.
model = YOLO("yolov8x")

results = model.train(
    data="/content/yolocustom.yaml",
    imgsz=(1024, 1024),
    epochs=50,
    batch=BATCH_SIZE,
    patience=5,
    workers=16,
    device=0,
    exist_ok=True,
    project=f"{MODEL}",
    name="train",
    seed=SEED,
    pretrained=False,
    resume=True,
    optimizer="Adam",
    lr0=1e-3,
    augment=True,
    val=True,
    cache=True
)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def get_test_image_paths(test_image_paths):
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [ ]:
model = YOLO("v2/train/weights/best.pt")
test_image_paths = glob("../data/yolo/test/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(1024, 1024), iou=0.2, conf=0.5, save_conf=True, save=False, save_txt=True, project=f"{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

In [ ]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]

    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)

    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [ ]:
infer_txts = glob(f"{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"../data/test/{base_file_name}.png").shape[:2]
    with open(infer_txt, "r") as reader:
        lines = reader.readlines()
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"../results/{MODEL}.csv", index=False)